# <font color='blue'>Data Science Challenge @ ITA 2022</font>
# <font color='blue'>Equipe DIOMGIS</font>

## <font color='blue'>1º Fase</font>

### <font color='blue'>Predição de pregões futuros de ativos que compõem o índice SP500.</font>

![title](data\image\logo.jpeg)

In [14]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.9.12


In [15]:
# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.

!pip install -q -U watermark

In [16]:
# Bibliotecas e Frameworks

import numpy as np
import pandas as pd
import pandas_datareader.data as web
import tensorflow as tf
from tensorflow import keras
from keras.layers import LSTM, Dense, Dropout
from keras.models import Sequential, load_model
from keras.optimizers import *
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau, TerminateOnNaN
from keras.losses import MeanSquaredError
from tensorboard import notebook
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from time import time
from datetime import datetime
import os

In [17]:
# Versões dos pacotes usados neste jupyter notebook

%reload_ext watermark
%watermark -a "Equipe DIOMGIS" --iversions

Author: Equipe DIOMGIS

pandas_datareader: 0.10.0
keras            : 2.10.0
seaborn          : 0.11.2
matplotlib       : 3.5.2
tensorboard      : 2.10.1
pandas           : 1.4.3
numpy            : 1.21.5
tensorflow       : 2.10.0



In [18]:
sns.set_style('whitegrid')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
%matplotlib inline
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)
plt.rcParams['figure.figsize'] = (15, 10)

In [6]:
#Confirma se o TensorFlow pode acessar a GPU

device_name = tf.test.gpu_device_name()
if not device_name:
    raise SystemError('GPU device not found')
    
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [7]:
# Estado da GPU

!nvidia-smi

Tue Oct 18 20:30:53 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.94       Driver Version: 516.94       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:65:00.0  On |                  N/A |
|  0%   50C    P2    29W / 220W |    806MiB /  8192MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
# Parametros fixos de treinamento

verbose = 2
seed = 25
steps = 30
epochs = 2000
batch_size = 32
logRetPeriod = 20
graphic = True
downloadData = True
trainModel = True

np.random.seed(seed)

In [23]:
forecast = ['2022-10-24', '2022-10-25', '2022-10-26', '2022-10-27', '2022-10-28', 
            '2022-10-31', '2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04', 
            '2022-11-07', '2022-11-08', '2022-11-09', '2022-11-10', '2022-11-11',
            '2022-11-14', '2022-11-15', '2022-11-16', '2022-11-17', '2022-11-18']

In [24]:

ativos = ['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT',
          'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES',
          'AFL', 'AIG', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL',
          'ALLE', 'AMAT', 'AMCR', 'AMD', 'AME', 'AMGN', 'AMP', 'AMT', 'AMZN',
          'ANET', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'APTV', 'ARE',
          'ATO', 'ATVI', 'AVB', 'AVGO', 'AVY', 'AWK', 'AXP', 'AZO', 'BA',
          'BAC', 'BALL', 'BAX', 'BBWI', 'BBY', 'BDX', 'BEN', 'BF.B', 'BIIB',
          'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BMY', 'BR', 'BRK.B', 'BRO',
          'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CARR', 'CAT', 'CB',
          'CBOE', 'CBRE', 'CCI', 'CCL', 'CDAY', 'CDNS', 'CDW', 'CE', 'CEG',
          'CF', 'CFG', 'CHD', 'CHRW', 'CHTR', 'CI', 'CINF', 'CL', 'CLX',
          'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'COF',
          'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CPT', 'CRL', 'CRM', 'CSCO',
          'CSGP', 'CSX', 'CTAS', 'CTLT', 'CTRA', 'CTSH', 'CTVA', 'CVS',
          'CVX', 'CZR', 'D', 'DAL', 'DD', 'DE', 'DFS', 'DG', 'DGX', 'DHI',
          'DHR', 'DIS', 'DISH', 'DLR', 'DLTR', 'DOV', 'DOW', 'DPZ', 'DRI',
          'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'DXCM', 'EA', 'EBAY', 'ECL',
          'ED', 'EFX', 'EIX', 'EL', 'ELV', 'EMN', 'EMR', 'ENPH', 'EOG',
          'EPAM', 'EQIX', 'EQR', 'EQT', 'ES', 'ESS', 'ETN', 'ETR', 'ETSY',
          'EVRG', 'EW', 'EXC', 'EXPD', 'EXPE', 'EXR', 'F', 'FANG', 'FAST',
          'FBHS', 'FCX', 'FDS', 'FDX', 'FE', 'FFIV', 'FIS', 'FISV', 'FITB',
          'FLT', 'FMC', 'FOX', 'FOXA', 'FRC', 'FRT', 'FTNT', 'FTV', 'GD',
          'GE', 'GILD', 'GIS', 'GL', 'GLW', 'GM', 'GNRC', 'GOOG', 'GOOGL',
          'GPC', 'GPN', 'GRMN', 'GS', 'GWW', 'HAL', 'HAS', 'HBAN', 'HCA',
          'HD', 'HES', 'HIG', 'HII', 'HLT', 'HOLX', 'HON', 'HPE', 'HPQ',
          'HRL', 'HSIC', 'HST', 'HSY', 'HUM', 'HWM', 'IBM', 'ICE', 'IDXX',
          'IEX', 'IFF', 'ILMN', 'INCY', 'INTC', 'INTU', 'INVH', 'IP', 'IPG',
          'IQV', 'IR', 'IRM', 'ISRG', 'IT', 'ITW', 'IVZ', 'J', 'JBHT', 'JCI',
          'JKHY', 'JNJ', 'JNPR', 'JPM', 'K', 'KDP', 'KEY', 'KEYS', 'KHC',
          'KIM', 'KLAC', 'KMB', 'KMI', 'KMX', 'KO', 'KR', 'L', 'LDOS', 'LEN',
          'LH', 'LHX', 'LIN', 'LKQ', 'LLY', 'LMT', 'LNC', 'LNT', 'LOW',
          'LRCX', 'LUMN', 'LUV', 'LVS', 'LW', 'LYB', 'LYV', 'MA', 'MAA',
          'MAR', 'MAS', 'MCD', 'MCHP', 'MCK', 'MCO', 'MDLZ', 'MDT', 'MET',
          'META', 'MGM', 'MHK', 'MKC', 'MKTX', 'MLM', 'MMC', 'MMM', 'MNST',
          'MO', 'MOH', 'MOS', 'MPC', 'MPWR', 'MRK', 'MRNA', 'MRO', 'MS',
          'MSCI', 'MSFT', 'MSI', 'MTB', 'MTCH', 'MTD', 'MU', 'NCLH', 'NDAQ',
          'NDSN', 'NEE', 'NEM', 'NFLX', 'NI', 'NKE', 'NLOK', 'NLSN', 'NOC',
          'NOW', 'NRG', 'NSC', 'NTAP', 'NTRS', 'NUE', 'NVDA', 'NVR', 'NWL',
          'NWS', 'NWSA', 'NXPI', 'O', 'ODFL', 'OGN', 'OKE', 'OMC', 'ON',
          'ORCL', 'ORLY', 'OTIS', 'OXY', 'PARA', 'PAYC', 'PAYX', 'PCAR',
          'PCG', 'PEAK', 'PEG', 'PEP', 'PFE', 'PFG', 'PG', 'PGR', 'PH',
          'PHM', 'PKG', 'PKI', 'PLD', 'PM', 'PNC', 'PNR', 'PNW', 'POOL',
          'PPG', 'PPL', 'PRU', 'PSA', 'PSX', 'PTC', 'PWR', 'PXD', 'PYPL',
          'QCOM', 'QRVO', 'RCL', 'RE', 'REG', 'REGN', 'RF', 'RHI', 'RJF',
          'RL', 'RMD', 'ROK', 'ROL', 'ROP', 'ROST', 'RSG', 'RTX', 'SBAC',
          'SBNY', 'SBUX', 'SCHW', 'SEDG', 'SEE', 'SHW', 'SIVB', 'SJM', 'SLB',
          'SNA', 'SNPS', 'SO', 'SPG', 'SPGI', 'SRE', 'STE', 'STT', 'STX',
          'STZ', 'SWK', 'SWKS', 'SYF', 'SYK', 'SYY', 'T', 'TAP', 'TDG',
          'TDY', 'TECH', 'TEL', 'TER', 'TFC', 'TFX', 'TGT', 'TJX', 'TMO',
          'TMUS', 'TPR', 'TRMB', 'TROW', 'TRV', 'TSCO', 'TSLA', 'TSN', 'TT',
          'TTWO', 'TWTR', 'TXN', 'TXT', 'TYL', 'UAL', 'UDR', 'UHS', 'ULTA',
          'UNH', 'UNP', 'UPS', 'URI', 'USB', 'V', 'VFC', 'VICI', 'VLO',
          'VMC', 'VNO', 'VRSK', 'VRSN', 'VRTX', 'VTR', 'VTRS', 'VZ', 'WAB',
          'WAT', 'WBA', 'WBD', 'WDC', 'WEC', 'WELL', 'WFC', 'WHR', 'WM',
          'WMB', 'WMT', 'WRB', 'WRK', 'WST', 'WTW', 'WY', 'WYNN', 'XEL',
          'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION', 'ZTS']

## Download dos Dados

In [25]:
if downloadData:

    start_date = "2017-10-21"
    end_date = "2022-10-21"

    data = web.DataReader(name = '^GSPC', data_source = 'yahoo', start = start_date, end = end_date)
    SP500_index = pd.DataFrame(data['Close']).reset_index().rename(columns={'Close': 'SP500', 'Date': 'Dia'})

    SP500_close = pd.DataFrame()

    for ativo in ativos:
  
        if ativo == 'BF.B':
            ativo = 'BF-B'

        if ativo == 'BRK.B':
            ativo = 'BRK-B'

        data = web.DataReader(name = ativo, data_source = 'yahoo', start = start_date, end = end_date)
        temp_close = pd.DataFrame(data['Close'])
        SP500_close = pd.concat([SP500_close, temp_close], axis = 1)

        
    SP500_close.columns = ativos
    SP500_close.reset_index(inplace = True)
    SP500_close.rename(columns={'Date': 'Dia'}, inplace = True)

    assert SP500_close.isna().sum().mean() == 0,  "Valores Faltantes"
    assert SP500_index.isna().sum().mean() == 0,  "Valores Faltantes"

    SP500_close.to_csv(path_or_buf = 'data/SP500_close', index = False)
    SP500_index.to_csv(path_or_buf = 'data/SP500_index', index = False)
    
else:
    
    SP500_close = pd.read_csv('data/SP500_close')
    SP500_index = pd.read_csv('data/SP500_index')

In [37]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

setores = {'Industrials': df.loc[df['GICS Sector'] == 'Industrials']['Symbol'].tolist(),
           'HealthCare': df.loc[df['GICS Sector'] == 'Health Care']['Symbol'].tolist(),
           'InformationTechnology': df.loc[df['GICS Sector'] == 'Information Technology']['Symbol'].tolist(),
           'CommunicationServices': df.loc[df['GICS Sector'] == 'Communication Services']['Symbol'].tolist(),
           'ConsumerStaples': df.loc[df['GICS Sector'] == 'Consumer Staples']['Symbol'].tolist(),
           'ConsumerDiscretionary': df.loc[df['GICS Sector'] == 'Consumer Discretionary']['Symbol'].tolist(),
           'Utilities': df.loc[df['GICS Sector'] == 'Utilities']['Symbol'].tolist(),
           'Financials': df.loc[df['GICS Sector'] == 'Financials']['Symbol'].tolist(),
           'Materials': df.loc[df['GICS Sector'] == 'Materials']['Symbol'].tolist(),
           'RealEstate': df.loc[df['GICS Sector'] == 'Real Estate']['Symbol'].tolist(),
           'Energy': df.loc[df['GICS Sector'] == 'Energy']['Symbol'].tolist()
          }

## Pré-Processamento dos Dados

In [13]:
def generatorTimeframeTable(table, ativo):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    
    TimeframeTable = pd.DataFrame(np.zeros((len(table[ativo])-steps, steps+1), dtype='float64'), columns = nameColumns)

    for index, close in enumerate(table[ativo]):
        tempA = index
        tempB = 0
        for i in range(steps+1):
            if tempA < len(table[ativo])-steps and tempA >=0:
                TimeframeTable.iloc[tempA, tempB] = close

            tempA -= 1
            tempB += 1

    timeIndex = table.iloc[steps:,0]
    TimeframeTable["Dia"] = timeIndex.to_numpy()
    TimeframeTable.set_index("Dia", inplace = True)
    
    return TimeframeTable

In [14]:
def createTrainScaler(df):
    
    trainScaler = pd.DataFrame()
 
    for _ in range(steps+1):
        temp_close = pd.DataFrame(df.iloc[:,-1])
        trainScaler = pd.concat([trainScaler, temp_close], axis = 1)

    scaler = StandardScaler()

    scaler.fit(trainScaler)

    return scaler

In [15]:
def preprocessingdata(steps, df, ativos):
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))
    

    aux = []
    
    for ativo in ativos:
        trainDataAtivo = generatorTimeframeTable(df, ativo)
        trainDataAtivo.dropna(axis = 0, inplace = True)
        
        #----Score-Z--------------------------------------
        scaler = createTrainScaler(trainDataAtivo)
        trainDataAtivo = scaler.transform(trainDataAtivo)
        #-------------------------------------------------
        aux.append(trainDataAtivo)
    
    trainData = np.concatenate(tuple(aux), axis=0)
    
    X = trainData[:, :-1]
    y = trainData[:, -1]
    

    #------Divisão de dados entre Treino e Validação------------------------------------------------
    
    X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size = 0.2, shuffle = False)

    X_treino = X_treino.reshape((-1, steps, 1))
    X_teste = X_teste.reshape((-1, steps, 1))
    #-----------------------------------------------------------------------------------------------
    
    return [X_treino, X_teste, y_treino, y_teste]

## Construção, Treinamento e Avaliação do Modelo

In [16]:
# Callbacks

tensorboard = TensorBoard(log_dir="logs/{}".format(datetime.now().strftime('%d-%B-%Ih%Mmin')))

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=20,
                          verbose = verbose,
                          restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='loss',
                              factor=0.2,
                              patience=3,
                              mode="min",
                              verbose = verbose,
                              min_delta=0.0001,
                              min_lr=0)

callbacks = [tensorboard, earlystop, reduce_lr, TerminateOnNaN()]

In [17]:
def create_model():
     
    model = Sequential()
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True,
                   input_shape = (steps, 1)))  

    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = True))  
    
    model.add(LSTM(160,
                   activation = 'tanh',
                   recurrent_activation = 'sigmoid',
                   return_sequences = False)) 
    
    model.add(Dense(1, activation = 'linear'))
    
    Lmse = MeanSquaredError()
    
    opt_Adadelta = Adadelta(learning_rate = 0.01, rho = 0.95, epsilon = 1e-07)

    model.compile(loss= Lmse, optimizer = opt_Adadelta)

    return model

In [18]:
def fillTableFrame(ativo, tablePrevision, model, table = SP500_close):

    TimeframeTable = generatorTimeframeTable(table, ativo)
    
    index_data = TimeframeTable.index
    
    scaler = createTrainScaler(TimeframeTable)

    TimeframeTable = scaler.transform(TimeframeTable)
    
    nameColumns = []

    for i in range(steps,-1,-1):
        nameColumns.append('Close-{}'.format(i))

    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    
    for day in forecast:
        
        current_info = TimeframeTable.iloc[-1, 1:].to_numpy()
        
        standardCurrentInfo = current_info.reshape(1, steps, 1).astype('float32')
        
        current_forecast = model.predict(standardCurrentInfo, verbose=False).reshape(1,)
        
        new_line = np.concatenate((current_info, current_forecast), axis = 0)
        
        TimeframeTable = pd.concat([TimeframeTable,
                                    pd.DataFrame(new_line.reshape(1, -1),
                                                 columns = nameColumns,
                                                 index = [day])], axis = 0)
        
        
    index_data = TimeframeTable.index  
    
    TimeframeTable = scaler.inverse_transform(TimeframeTable)
    
    TimeframeTable = pd.DataFrame(TimeframeTable, columns = nameColumns, index = index_data)
    
    TimeframeTable.index = pd.to_datetime(TimeframeTable.index)
    
    
    #--------Popula tabela de previsão---------------------------------------------------
    if ativo in ativos:
            for day in forecast:
                tablePrevision.loc[day, ativo] = TimeframeTable.loc[day, 'Close-0']
    #------------------------------------------------------------------------------------
   
    return TimeframeTable

In [19]:
# Popula tabela de log-Retorno e gera gráficos

def fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model):
    
    lengthTable = len(tableLogRet)
    
    time = datetime.now().strftime('%d-%B-%Ih%Mmin')

    for ativo in setor:

        TimeframeSPAux = fillTableFrame(ativo, tablePrevision, model)

        #-----------Graphic------------------------------------------------------------------------------------------
        if graphic:
            
            outdir = './graphics/{}-{}'.format(nameSetor, time)

            if not os.path.exists(outdir):
                os.mkdir(outdir)
            
            fig, ax = plt.subplots()
            ax.plot(TimeframeSPAux.index[:-len(forecast)], TimeframeSPAux.iloc[:-len(forecast), -1], linewidth=2.0, c = 'b')
            ax.plot(TimeframeSPAux.index[-len(forecast):], TimeframeSPAux.iloc[-len(forecast):, -1], linewidth=2.0, c = 'r', ls = '-')
            ax.legend(['Atual', 'Previsão'])
            ax.set_title('Preço de Fechamento - {}'.format(ativo))
            ax.set(xlabel='Tempo (ano)', ylabel='Preço ($)')
            nameGraphic = '{}.jpg'.format(ativo)
            fullname = os.path.join(outdir, nameGraphic)
            plt.savefig(fullname)
            plt.close(fig)
        #------------------------------------------------------------------------------------------------------------

        
        #--------Popula tabela de Log Retorno------------------------------------------------------------------------
        for n in range(len(forecast)):
            tableLogRet.loc[lengthTable-n-1, ativo] = \
            np.log(TimeframeSPAux.iloc[lengthTable-steps-n-1, -1] / TimeframeSPAux.iloc[lengthTable-steps-n-1-logRetPeriod, -1])
        #------------------------------------------------------------------------------------------------------------

In [20]:
# Cria tabela de Previsão

update = pd.DataFrame(index = pd.to_datetime(forecast), columns = ativos) \
    .reset_index().rename(columns={'index': 'Dia'})

tablePrevision = pd.concat([SP500_close, update], axis = 0, ignore_index = True).set_index('Dia')

tablePrevision.index = pd.to_datetime(tablePrevision.index)

In [21]:
# Cria tabela de Log-Retorno vazia 

index_data = pd.to_datetime(SP500_close['Dia'].append(pd.Series(forecast)))

tableLogRet = pd.DataFrame(index = index_data,
                           columns = ativos).reset_index().rename(columns={'index': 'Dia'})

tableLogRet['Dia'] = tableLogRet['Dia'].apply(lambda date: date.strftime('%d/%m'))

In [22]:
for nameSetor, setor in setores.items():

    if trainModel:
        X_treino, X_teste, y_treino, y_teste = preprocessingdata(steps, SP500_close, setor)
        model = create_model()

        model.fit(x = X_treino,
                  y = y_treino,
                  batch_size = batch_size,
                  epochs = epochs,
                  verbose = verbose,
                  validation_data = (X_teste, y_teste),
                  callbacks = callbacks)
        
        #---------Save Model---------------------------
        model.save('saveModel/{}/'.format(nameSetor),
                   overwrite=True,
                   include_optimizer=True,
                   save_format = 'tf')
        #----------------------------------------------

        scoreTrain = model.evaluate(X_treino, y_treino)
        scoreTest = model.evaluate(X_teste, y_teste)

        print('\n\nErro quadrático médio em dados de treinamento: {:.5f}\n\nErro quadrático médio em dados de teste: {:.5f}\n\n'\
            .format(scoreTrain, scoreTest))

        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)
        
    else:
        model = load_model('saveModel/{}/'.format(nameSetor))
        fillPrediction(tableLogRet, tablePrevision, nameSetor, setor, model)


Epoch 1/2000
2143/2143 - 46s - loss: 0.0881 - val_loss: 0.0458 - lr: 0.0100 - 46s/epoch - 21ms/step
Epoch 2/2000
2143/2143 - 36s - loss: 0.0434 - val_loss: 0.0376 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 3/2000
2143/2143 - 36s - loss: 0.0375 - val_loss: 0.0327 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 4/2000
2143/2143 - 36s - loss: 0.0332 - val_loss: 0.0292 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 5/2000
2143/2143 - 36s - loss: 0.0301 - val_loss: 0.0267 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 6/2000
2143/2143 - 36s - loss: 0.0280 - val_loss: 0.0251 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 7/2000
2143/2143 - 36s - loss: 0.0265 - val_loss: 0.0239 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 8/2000
2143/2143 - 36s - loss: 0.0253 - val_loss: 0.0228 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 9/2000
2143/2143 - 38s - loss: 0.0243 - val_loss: 0.0219 - lr: 0.0100 - 38s/epoch - 18ms/step
Epoch 10/2000
2143/2143 - 38s - loss: 0.0233 - val_loss: 0.0211 - lr: 0.0100 - 38s/epoch - 18ms/step

Epoch 83/2000
2143/2143 - 36s - loss: 0.0092 - val_loss: 0.0084 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 84/2000
2143/2143 - 35s - loss: 0.0092 - val_loss: 0.0083 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 85/2000
2143/2143 - 36s - loss: 0.0092 - val_loss: 0.0084 - lr: 0.0100 - 36s/epoch - 17ms/step
Epoch 86/2000
2143/2143 - 35s - loss: 0.0091 - val_loss: 0.0083 - lr: 0.0100 - 35s/epoch - 16ms/step
Epoch 87/2000
2143/2143 - 35s - loss: 0.0091 - val_loss: 0.0083 - lr: 0.0100 - 35s/epoch - 17ms/step
Epoch 88/2000

Epoch 88: ReduceLROnPlateau reducing learning rate to 0.0019999999552965165.
2143/2143 - 35s - loss: 0.0091 - val_loss: 0.0082 - lr: 0.0100 - 35s/epoch - 16ms/step
Epoch 89/2000
2143/2143 - 37s - loss: 0.0090 - val_loss: 0.0082 - lr: 0.0020 - 37s/epoch - 17ms/step
Epoch 90/2000
2143/2143 - 37s - loss: 0.0090 - val_loss: 0.0082 - lr: 0.0020 - 37s/epoch - 17ms/step
Epoch 91/2000
2143/2143 - 38s - loss: 0.0090 - val_loss: 0.0082 - lr: 0.0020 - 38s/epoch - 18ms/step
Epoch 92/2000

INFO:tensorflow:Assets written to: saveModel/Industrials/assets


INFO:tensorflow:Assets written to: saveModel/Industrials/assets


536/536 [==============================] - 4s 7ms/step - loss: 0.0081


Erro quadrático médio em dados de treinamento: 0.00898

Erro quadrático médio em dados de teste: 0.00814


Epoch 1/2000
1929/1929 - 42s - loss: 0.1068 - val_loss: 0.0487 - lr: 0.0100 - 42s/epoch - 22ms/step
Epoch 2/2000
1929/1929 - 34s - loss: 0.0573 - val_loss: 0.0402 - lr: 0.0100 - 34s/epoch - 18ms/step
Epoch 3/2000
1929/1929 - 34s - loss: 0.0501 - val_loss: 0.0356 - lr: 0.0100 - 34s/epoch - 18ms/step
Epoch 4/2000
1929/1929 - 34s - loss: 0.0450 - val_loss: 0.0320 - lr: 0.0100 - 34s/epoch - 18ms/step
Epoch 5/2000
1929/1929 - 34s - loss: 0.0410 - val_loss: 0.0292 - lr: 0.0100 - 34s/epoch - 18ms/step
Epoch 6/2000
1929/1929 - 35s - loss: 0.0381 - val_loss: 0.0273 - lr: 0.0100 - 35s/epoch - 18ms/step
Epoch 7/2000
1929/1929 - 34s - loss: 0.0360 - val_loss: 0.0257 - lr: 0.0100 - 34s/epoch - 18ms/step
Epoch 8/2000
1929/1929 - 35s - loss: 0.0343 - val_loss: 0.0244 - lr: 0.0100 - 35s/epoch - 18ms/step
Epoch 9/2000
1929/192

...

Epoch 82/2000
948/948 - 16s - loss: 0.0147 - val_loss: 0.0162 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 83/2000
948/948 - 16s - loss: 0.0146 - val_loss: 0.0161 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 84/2000
948/948 - 16s - loss: 0.0145 - val_loss: 0.0160 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 85/2000
948/948 - 16s - loss: 0.0144 - val_loss: 0.0159 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 86/2000
948/948 - 16s - loss: 0.0143 - val_loss: 0.0157 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 87/2000
948/948 - 16s - loss: 0.0143 - val_loss: 0.0157 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 88/2000
948/948 - 16s - loss: 0.0142 - val_loss: 0.0155 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 89/2000
948/948 - 16s - loss: 0.0141 - val_loss: 0.0154 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 90/2000
948/948 - 16s - loss: 0.0140 - val_loss: 0.0153 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 91/2000
948/948 - 16s - loss: 0.0139 - val_loss: 0.0152 - lr: 0.0100 - 16s/epoch - 17ms/step
Epoch 92/2

Epoch 160/2000
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 3.2000e-06 - 16s/epoch - 17ms/step
Epoch 161/2000

Epoch 161: ReduceLROnPlateau reducing learning rate to 6.399999165296323e-07.
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 3.2000e-06 - 16s/epoch - 17ms/step
Epoch 162/2000
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 6.4000e-07 - 16s/epoch - 17ms/step
Epoch 163/2000
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 6.4000e-07 - 16s/epoch - 17ms/step
Epoch 164/2000

Epoch 164: ReduceLROnPlateau reducing learning rate to 1.2799998785339995e-07.
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 6.4000e-07 - 16s/epoch - 17ms/step
Epoch 165/2000
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 1.2800e-07 - 16s/epoch - 17ms/step
Epoch 166/2000
948/948 - 16s - loss: 0.0110 - val_loss: 0.0118 - lr: 1.2800e-07 - 16s/epoch - 17ms/step
Epoch 167/2000

Epoch 167: ReduceLROnPlateau reducing learning rate to 2.5599996433811613e-08.
948/948 - 16s - lo

INFO:tensorflow:Assets written to: saveModel/RealEstate/assets


INFO:tensorflow:Assets written to: saveModel/RealEstate/assets


237/237 [==============================] - 2s 7ms/step - loss: 0.0118


Erro quadrático médio em dados de treinamento: 0.01097

Erro quadrático médio em dados de teste: 0.01177




# Resultados

### Salva Métricas

In [23]:
# Salva tabela de previsão

outdirP = './previsao/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirP):
    os.mkdir(outdirP)

fullnameP = os.path.join(outdirP, 'previsao.csv')

tablePrevision.to_csv(fullnameP, index = True, decimal = '.', sep=',')  

In [24]:
# Salva tabela Entregável do Log-Retorno no padrão

outdirLR = './logRetorno/{}'.format(datetime.now().strftime('%d-%B-%Ih%Mmin'))

if not os.path.exists(outdirLR):
    os.mkdir(outdirLR)

fullnameLR = os.path.join(outdirLR, 'logRetorno.csv')

tableLogRet.iloc[-len(forecast):, :].to_csv(fullnameLR, index = False, decimal = '.', sep=',')       